In [1]:
from datasets import load_dataset
import pandas as pd
mbpp = load_dataset("mbpp")

No config specified, defaulting to: mbpp/full
Found cached dataset mbpp (/home/chloe/.cache/huggingface/datasets/mbpp/full/1.0.2/4458a31cd4305553c8e88e3f0bfb94fc74fe1a9faeeb8c32ed166939735eaeff)


  0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
mbpp_train_df = pd.DataFrame(mbpp["train"])
mbpp_train_df.describe()

,task_id
count,374.000000
mean,787.500000
std,108.108742
min,601.000000
25%,694.250000
50%,787.500000
75%,880.750000
max,974.000000


In [3]:
mbpp_prompt_df = pd.DataFrame(mbpp["prompt"])
mbpp_test_df = pd.DataFrame(mbpp["test"])
print(mbpp_prompt_df.iloc[3]['text'])
print(mbpp_test_df.iloc[3]['test_list'])

Write a function to find the largest integers from a given list of numbers using heap queue algorithm.
['assert find_Volume(10,8,6) == 240', 'assert find_Volume(3,2,2) == 6', 'assert find_Volume(1,2,1) == 1']


In [4]:
mbpp_test_df.describe()

,task_id
count,500.000000
mean,260.500000
std,144.481833
min,11.000000
25%,135.750000
50%,260.500000
75%,385.250000
max,510.000000


In [8]:
print(mbpp_train_df.iloc[8]['code'])

def floor_Min(A,B,N):
    x = max(B - 1,N)
    return (A*x) // B


In [10]:
mbpp_train_df['code'].value_counts

<bound method IndexOpsMixin.value_counts of 0      class Pair(object): \r\n\tdef __init__(self, a...
1      def first_repeated_char(str1):\r\n  for index,...
2      def get_ludic(n):\r\n\tludics = []\r\n\tfor i ...
3      def reverse_words(s):\r\n        return ' '.jo...
4      def prime_num(num):\r\n  if num >=1:\r\n   for...
                             ...                        
369    def min_of_two( x, y ):\r\n    if x < y:\r\n  ...
370    def maximum_segments(n, a, b, c) : \r\n\tdp = ...
371    def concatenate_nested(test_tup1, test_tup2):\...
372    def left_rotate(s,d):\r\n    tmp = s[d : ] + s...
373    def min_sum_path(A): \r\n\tmemo = [None] * len...
Name: code, Length: 374, dtype: object>

In [19]:
max_list_length = mbpp_test_df['test_list'].apply(len).max()
max_list_length

3

In [4]:
# format the train df such that it uses the same format as the coqllm dataset
mbpp_train_df[['test_case_1', 'test_case_2', 'test_case_3']] = mbpp_train_df['test_list'].apply(
    lambda x: pd.Series(x[:3] + [None] * max(0, 3 - len(x)))
)
mbpp_train_coq = mbpp_train_df[['text', 'test_case_1', 'test_case_2', 'test_case_3']]
mbpp_train_coq = mbpp_train_coq.rename(columns={'text': 'specification'})
# remove "assert" statements from the test cases
test_case_columns = ['test_case_1', 'test_case_2', 'test_case_3']
def remove_asserts(text):
    if text is None:
        return None
    return text.replace('assert ', '')
def fix_equals(text):
    if text is None:
        return None
    return text.replace('==', '=')
def change_language(text):
    if text is None:
        return None
    return text.replace('Write a python function', 'Write a Coq function')

mbpp_train_coq[test_case_columns] = mbpp_train_coq[test_case_columns].applymap(remove_asserts)
mbpp_train_coq[test_case_columns] = mbpp_train_coq[test_case_columns].applymap(fix_equals)
mbpp_train_coq[['specification']] = mbpp_train_coq[['specification']].applymap(change_language)

mbpp_train_coq.head()

,specification,test_case_1,test_case_2,test_case_3
0,Write a function to find the longest chain whi...,"max_chain_length([Pair(5, 24), Pair(15, 25),Pa...","max_chain_length([Pair(1, 2), Pair(3, 4),Pair(...","max_chain_length([Pair(19, 10), Pair(11, 12),P..."
1,Write a Coq function to find the first repeate...,"first_repeated_char(""abcabc"") = ""a""","first_repeated_char(""abc"") = ""None""","first_repeated_char(""123123"") = ""1"""
2,Write a function to get a lucid number smaller...,"get_ludic(10) = [1, 2, 3, 5, 7]","get_ludic(25) = [1, 2, 3, 5, 7, 11, 13, 17, 23...","get_ludic(45) = [1, 2, 3, 5, 7, 11, 13, 17, 23..."
3,Write a function to reverse words in a given s...,"reverse_words(""python program"")=(""program pyth...","reverse_words(""java language"")=(""language java"")","reverse_words(""indian man"")=(""man indian"")"
4,Write a function to check if the given integer...,prime_num(13)=True,prime_num(7)=True,prime_num(-1010)=False


In [5]:
mbpp_train_coq.to_csv('MBPP_Coq_Train.csv', index=False)

In [24]:
# format the test df such that it uses the same format as the coqllm dataset
mbpp_test_df[['test_case_1', 'test_case_2', 'test_case_3']] = mbpp_test_df['test_list'].apply(
    lambda x: pd.Series(x[:3] + [None] * max(0, 3 - len(x)))
)
mbpp_test_coq = mbpp_test_df[['text', 'test_case_1', 'test_case_2', 'test_case_3']]
mbpp_test_coq = mbpp_test_coq.rename(columns={'text': 'specification'})
# remove "assert" statements from the test cases
test_case_columns = ['test_case_1', 'test_case_2', 'test_case_3']
def remove_asserts(text):
    if text is None:
        return None
    return text.replace('assert ', '')
def fix_equals(text):
    if text is None:
        return None
    return text.replace('==', '=')
def change_language(text):
    if text is None:
        return None
    return text.replace('Write a python function', 'Write a Coq function')

mbpp_test_coq[test_case_columns] = mbpp_test_coq[test_case_columns].applymap(remove_asserts)
mbpp_test_coq[test_case_columns] = mbpp_test_coq[test_case_columns].applymap(fix_equals)
mbpp_test_coq[['specification']] = mbpp_test_coq[['specification']].applymap(change_language)

mbpp_test_coq.head()


,specification,test_case_1,test_case_2,test_case_3
0,Write a Coq function to remove first and last ...,"remove_Occ(""hello"",""l"") = ""heo""","remove_Occ(""abcda"",""a"") = ""bcd""","remove_Occ(""PHP"",""P"") = ""H"""
1,Write a function to sort a given matrix in asc...,"sort_matrix([[1, 2, 3], [2, 4, 5], [1, 1, 1]])...","sort_matrix([[1, 2, 3], [-2, 4, -5], [1, -1, 1...","sort_matrix([[5,8,9],[6,4,3],[2,1,4]])=[[2, 1,..."
2,Write a function to count the most common word...,"count_common(['red','green','black','pink','bl...","count_common(['one', 'two', 'three', 'four', '...","count_common(['Facebook', 'Apple', 'Amazon', '..."
3,Write a Coq function to find the volume of a t...,"find_Volume(10,8,6) = 240","find_Volume(3,2,2) = 6","find_Volume(1,2,1) = 1"
4,Write a function to split a string at lowercas...,"split_lowerstring(""AbCd"")=['bC','d']","split_lowerstring(""Python"")=['y', 't', 'h', 'o...","split_lowerstring(""Programming"")=['r', 'o', 'g..."


In [25]:
mbpp_test_coq.to_csv('MBPP_Coq_Test.csv', index=False)

# Formatting Function Test Cases

In [4]:
import pandas as pd
mbpp_train_coq = pd.read_csv('MBPP_Coq_Train.csv')

In [22]:
import re
def get_function_name(s):
    # Match the function name followed by an opening parenthesis
    match = re.match(r'(\w+)\(', s)
    
    if match:
        function_name = match.group(1)
        return function_name
    else:
        return None
    
def get_function_arguments(s):
    # Match the opening parenthesis, arguments, and closing parenthesis
    match = re.search(r'\((.*?)\)', s)
    
    if match:
        arguments = match.group(1)
        return arguments.split(',')
    else:
        return None
# def extract_function_and_args(s):
#     # Match function name and arguments
#     match = re.match(r'(\w+)"(.*?)","(.*?)"', s)
    
#     if match:
#         function_name = match.group(1)
#         arg1 = match.group(2)
#         arg2 = match.group(3)
        
#         return function_name, arg1, arg2
#     else:
#         return None, None, None

In [29]:
import ast
import re

# Define the Pair class
class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return f"Pair({self.x}, {self.y})"

def get_function_info(s):
    # Split the input string on the equal sign to get the function call part
    function_call_part = s.split('=')[0].strip()

    # Parse the function call part into an AST
    node = ast.parse(function_call_part)

    # Check if the first node is an Expr (expression)
    if isinstance(node.body[0], ast.Expr):
        # Check if the value is a Call (function call)
        if isinstance(node.body[0].value, ast.Call):
            call = node.body[0].value

            # Get the function name
            function_name = call.func.id

            # Get the function arguments as a list of strings
            raw_arguments = re.findall(r'\(.*?\)', function_call_part)[0][1:-1].split(',')
            # Evaluate the arguments as Python values and/or custom objects
            function_arguments = [eval(arg.strip()) for arg in raw_arguments]

            return function_name, function_arguments

    return None, None

# Example usage:
input_str = 'max_chain_length([Pair(5, 24), Pair(15, 25),Pair(27, 40), Pair(50, 60)], 4) = 3'
function_name, function_arguments = get_function_info(input_str)
print(function_name)  # Output: 'max_chain_length'
print(function_arguments)  # Output: [[Pair(5, 24), Pair(15, 25), Pair(27, 40), Pair(50, 60)], 4]


SyntaxError: unexpected EOF while parsing (<string>, line 1)

In [27]:
row = mbpp_train_coq.iloc[0]
print(row['test_case_1'])
s = row['test_case_1']
print(get_function_name(s))
print(get_function_arguments(s))
print(get_function_info(s))

max_chain_length([Pair(5, 24), Pair(15, 25),Pair(27, 40), Pair(50, 60)], 4) = 3
max_chain_length
['[Pair(5', ' 24']


ValueError: malformed node or string: <ast.Call object at 0x7f2a46816970>

In [11]:
for index, row in mbpp_train_coq.iterrows():
    print(extract_function_and_args(row['test_case_1']))

(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None, None)
(None, None,